## 1. Setup and Imports

In [ ]:
import os
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
from pathlib import Path
import json
from datetime import datetime

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Configuration

Adjust these paths and hyperparameters based on your setup.

In [ ]:
# Data paths (update these to your file locations)
DATA_CONFIG = {
    'train_jsonl': '../data/cora/cora_train_node_data.jsonl',
    'test_jsonl': '../data/cora/cora_test_node_data.jsonl',
    'embedding_path': '../data/cora/multi_hop_graph_embedding.pt',
}

# Model configuration
MODEL_CONFIG = {
    'llama_model': 'meta-llama/Meta-Llama-3-8B-Instruct',
    'graph_embedding_dim': 2048,
    'projector_hidden_dim': 4096,
    'num_hops': 5,
}

# Training configuration
TRAINING_CONFIG = {
    'batch_size': 8,
    'gradient_accumulation_steps': 4,  # Effective batch size = 8 * 4 = 32
    'learning_rate': 2e-4,
    'num_epochs': 10,
    'warmup_steps': 100,
    'num_workers': 4,
    'output_dir': './checkpoints',
    'save_every': 1,  # Save every N epochs
}

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Configuration:")
print("\nData:")
for k, v in DATA_CONFIG.items():
    print(f"  {k}: {v}")
print("\nModel:")
for k, v in MODEL_CONFIG.items():
    print(f"  {k}: {v}")
print("\nTraining:")
for k, v in TRAINING_CONFIG.items():
    print(f"  {k}: {v}")
print(f"\nDevice: {device}")

## 3. Load Model and Data

This will load the GWM-E model with frozen LLaMA and trainable projector.

In [ ]:
# Import model and dataset classes
from model import GWM_E
from dataset import create_dataloaders

print("="*70)
print(" "*20 + "Loading GWM-E Model")
print("="*70)

# Initialize model
model = GWM_E(
    llama_model_path=MODEL_CONFIG['llama_model'],
    graph_embedding_dim=MODEL_CONFIG['graph_embedding_dim'],
    projector_hidden_dim=MODEL_CONFIG['projector_hidden_dim'],
    num_hops=MODEL_CONFIG['num_hops'],
    freeze_llm=True,  # Only train projector
)

print("\n✓ Model loaded successfully!")
print(f"  LLaMA parameters: {sum(p.numel() for p in model.llm.parameters()) / 1e9:.2f}B (frozen)")
print(f"  Projector parameters: {sum(p.numel() for p in model.projector.parameters()) / 1e6:.2f}M (trainable)")

In [ ]:
print("="*70)
print(" "*20 + "Loading Datasets")
print("="*70)

# Create dataloaders
train_loader, test_loader = create_dataloaders(
    train_jsonl=DATA_CONFIG['train_jsonl'],
    test_jsonl=DATA_CONFIG['test_jsonl'],
    embedding_path=DATA_CONFIG['embedding_path'],
    tokenizer=model.tokenizer,
    batch_size=TRAINING_CONFIG['batch_size'],
    num_workers=TRAINING_CONFIG['num_workers'],
    num_hops=MODEL_CONFIG['num_hops'],
)

print(f"\n✓ Datasets loaded successfully!")
print(f"  Training samples: {len(train_loader.dataset):,}")
print(f"  Test samples: {len(test_loader.dataset):,}")
print(f"  Training batches: {len(train_loader):,}")
print(f"  Test batches: {len(test_loader):,}")

## 4. Setup Training

Configure optimizer and learning rate scheduler.

In [ ]:
print("="*70)
print(" "*20 + "Setting up Training")
print("="*70)

# Create output directory
output_dir = Path(TRAINING_CONFIG['output_dir'])
output_dir.mkdir(parents=True, exist_ok=True)

# Save training config
config_path = output_dir / "training_config.json"
all_config = {**DATA_CONFIG, **MODEL_CONFIG, **TRAINING_CONFIG}
with open(config_path, 'w') as f:
    json.dump(all_config, f, indent=2)
print(f"✓ Saved config to: {config_path}")

# Setup optimizer (only for projector parameters)
optimizer = torch.optim.AdamW(
    model.projector.parameters(),
    lr=TRAINING_CONFIG['learning_rate'],
    weight_decay=0.01,
)
print(f"✓ Optimizer: AdamW (lr={TRAINING_CONFIG['learning_rate']})")

# Calculate total training steps
total_steps = len(train_loader) * TRAINING_CONFIG['num_epochs'] // TRAINING_CONFIG['gradient_accumulation_steps']
print(f"✓ Total training steps: {total_steps:,}")

# Setup learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=TRAINING_CONFIG['warmup_steps'],
    num_training_steps=total_steps,
)
print(f"✓ Scheduler: Linear warmup ({TRAINING_CONFIG['warmup_steps']} steps) + decay")

print(f"\n✓ Training setup complete!")

## 5. Training Functions

Define training and evaluation functions.

In [ ]:
def train_epoch(model, train_loader, optimizer, scheduler, epoch, gradient_accumulation_steps, device):
    """
    Train for one epoch.
    
    Returns:
        Average loss for the epoch
    """
    model.train()
    total_loss = 0
    num_batches = 0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}")
    
    for batch_idx, batch in enumerate(progress_bar):
        # Move to device
        multi_hop_embedding = batch['multi_hop_embedding'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        logits, loss = model(
            multi_hop_embeddings=multi_hop_embedding,
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        
        # Scale loss for gradient accumulation
        loss = loss / gradient_accumulation_steps
        
        # Backward pass
        loss.backward()
        
        # Update weights every gradient_accumulation_steps
        if (batch_idx + 1) % gradient_accumulation_steps == 0:
            # Clip gradients
            torch.nn.utils.clip_grad_norm_(model.projector.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        # Track loss
        total_loss += loss.item() * gradient_accumulation_steps
        num_batches += 1
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': f"{loss.item() * gradient_accumulation_steps:.4f}",
            'lr': f"{scheduler.get_last_lr()[0]:.2e}"
        })
    
    return total_loss / num_batches


def evaluate(model, test_loader, device):
    """
    Evaluate the model on test set.
    
    Returns:
        avg_loss, accuracy
    """
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            # Move to device
            multi_hop_embedding = batch['multi_hop_embedding'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass
            logits, loss = model(
                multi_hop_embeddings=multi_hop_embedding,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            
            total_loss += loss.item()
            
            # Calculate accuracy (on non-masked tokens)
            mask = labels != -100
            predictions = logits.argmax(dim=-1)
            correct += ((predictions == labels) & mask).sum().item()
            total += mask.sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total if total > 0 else 0
    
    return avg_loss, accuracy


print("✓ Training functions defined")

## 6. Training Loop

Run the main training loop.

In [ ]:
print("="*70)
print(" "*25 + "STARTING TRAINING")
print("="*70)

best_accuracy = 0
training_history = []

for epoch in range(1, TRAINING_CONFIG['num_epochs'] + 1):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch}/{TRAINING_CONFIG['num_epochs']}")
    print(f"{'='*70}")
    
    # Train
    train_loss = train_epoch(
        model=model,
        train_loader=train_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        epoch=epoch,
        gradient_accumulation_steps=TRAINING_CONFIG['gradient_accumulation_steps'],
        device=device,
    )
    
    # Evaluate
    test_loss, test_accuracy = evaluate(
        model=model,
        test_loader=test_loader,
        device=device,
    )
    
    # Log results
    print(f"\n{'='*70}")
    print(f"Epoch {epoch} Results:")
    print(f"{'='*70}")
    print(f"  Train Loss:     {train_loss:.4f}")
    print(f"  Test Loss:      {test_loss:.4f}")
    print(f"  Test Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    
    # Save history
    training_history.append({
        'epoch': epoch,
        'train_loss': train_loss,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
    })
    
    # Save checkpoint
    if epoch % TRAINING_CONFIG['save_every'] == 0 or test_accuracy > best_accuracy:
        checkpoint_path = output_dir / f"projector_epoch_{epoch}.pt"
        model.save_projector(str(checkpoint_path))
        print(f"  ✓ Saved checkpoint: {checkpoint_path.name}")
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_path = output_dir / "projector_best.pt"
            model.save_projector(str(best_path))
            print(f"  ⭐ New best accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")

# Save training history
history_path = output_dir / "training_history.json"
with open(history_path, 'w') as f:
    json.dump(training_history, f, indent=2)

print("\n" + "="*70)
print(" "*20 + "✅ TRAINING COMPLETED!")
print("="*70)
print(f"Best test accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"Checkpoints saved to: {output_dir}")
print(f"Training history saved to: {history_path}")
print("="*70)

## 7. Visualize Training Progress

In [ ]:
import matplotlib.pyplot as plt

# Extract metrics
epochs = [h['epoch'] for h in training_history]
train_losses = [h['train_loss'] for h in training_history]
test_losses = [h['test_loss'] for h in training_history]
test_accuracies = [h['test_accuracy'] for h in training_history]

# Create plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
ax1.plot(epochs, train_losses, 'b-o', label='Train Loss')
ax1.plot(epochs, test_losses, 'r-o', label='Test Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Test Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy plot
ax2.plot(epochs, [acc * 100 for acc in test_accuracies], 'g-o', label='Test Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Test Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Saved training curves to: {output_dir / 'training_curves.png'}")

## 8. Summary Statistics

In [ ]:
print("="*70)
print(" "*25 + "TRAINING SUMMARY")
print("="*70)

print(f"\n📊 Final Results:")
print(f"  Best Test Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"  Final Train Loss:   {training_history[-1]['train_loss']:.4f}")
print(f"  Final Test Loss:    {training_history[-1]['test_loss']:.4f}")

print(f"\n📈 Training Progress:")
print(f"  Initial Accuracy: {training_history[0]['test_accuracy']:.4f} ({training_history[0]['test_accuracy']*100:.2f}%)")
print(f"  Final Accuracy:   {training_history[-1]['test_accuracy']:.4f} ({training_history[-1]['test_accuracy']*100:.2f}%)")
print(f"  Improvement:      {(training_history[-1]['test_accuracy'] - training_history[0]['test_accuracy']):.4f} ({(training_history[-1]['test_accuracy'] - training_history[0]['test_accuracy'])*100:.2f}%)")

print(f"\n💾 Saved Files:")
print(f"  • Best model: {output_dir / 'projector_best.pt'}")
print(f"  • Training history: {output_dir / 'training_history.json'}")
print(f"  • Training curves: {output_dir / 'training_curves.png'}")
print(f"  • Config: {output_dir / 'training_config.json'}")

print(f"\n🚀 Next Steps:")
print(f"  1. Run inference notebook to evaluate on test set")
print(f"  2. Or run: python inference.py \\")
print(f"       --test_jsonl {DATA_CONFIG['test_jsonl']} \\")
print(f"       --embedding_path {DATA_CONFIG['embedding_path']} \\")
print(f"       --projector_checkpoint {output_dir / 'projector_best.pt'}")

print("\n" + "="*70)

## Optional: Test a Single Prediction

Quick test to see model predictions.

In [ ]:
# Get a sample from test set
test_sample = test_loader.dataset[0]
test_conv = test_loader.dataset.conversations[0]

print("Testing model on a single example...\n")
print("="*70)
print("Sample Input:")
print("="*70)
for turn in test_conv['conversations']:
    role = turn['from']
    message = turn['value'][:200] + "..." if len(turn['value']) > 200 else turn['value']
    print(f"{role.upper()}: {message}\n")

# Prepare inputs
multi_hop_emb = test_sample['multi_hop_embedding'].unsqueeze(0).to(device)
input_ids = test_sample['input_ids'].unsqueeze(0).to(device)
attention_mask = test_sample['attention_mask'].unsqueeze(0).to(device)

# Generate prediction
model.eval()
with torch.no_grad():
    outputs = model.generate(
        multi_hop_embeddings=multi_hop_emb,
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=50,
        temperature=0.1,
    )

# Decode
input_length = input_ids.shape[1]
generated = model.tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)

print("="*70)
print("Model Prediction:")
print("="*70)
print(f"ASSISTANT: {generated}\n")
print("="*70)